In [115]:
# connect to my google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
# Install libraries for web scraping
# selenium for browser automation
# chromium-chromedriver for Selenium to control Chrome
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

/usr/lib/python3.11/pathlib.py:542: RuntimeWarning: coroutine 'main' was never awaited
  self._str = self._format_parsed_parts(self._drv, self._root,
/usr/lib/python3.11/pathlib.py:540: RuntimeWarning: coroutine 'main' was never awaited
  return self._str


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (156 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [117]:
# imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import asyncio
import aiohttp

In [118]:
# get the URL for the Premier League standings page on FBref
prem_standings_url = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"

In [119]:
# Configure Chrome for a headless browser due to Google Colab
# Headless mode == browser runs without a visible UI
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Use the driver to get the page data
driver.get(prem_standings_url)

# Wait for page to load and bypass Cloudflare
# waits up to 10 seconds for 'stats_table' element to ve visible
try:
    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "stats_table"))
    )
    print("Page loaded successfully.")
except Exception as e:
    print(f"An error occurred while waiting for the page to load: {e}")
    driver.quit()
    exit()

Page loaded successfully.


In [120]:
# use requests to get the html info
page_data = driver.page_source

Parsing HTML and Extracting Data

In [121]:
# want to use standings to get to the team's url page for indepth stats
soup = BeautifulSoup(page_data, 'html.parser')

# Select the table then select all the a tags
standindgs = soup.select('table.stats_table')[0]
a_tags = standindgs.find_all('a')
links = [l.get("href") for l in a_tags]
# We also get top goal scrorere and goalkeeper links which we don't want
links = [l for l in links if '/squads' in l]
# Make them absolute links
team_urls = [f"https://fbref.com{l}" for l in links]

In [122]:
# Practice using a specific team
chelsea_url = team_urls[3]
driver.get(chelsea_url)
time.sleep(5)

In [123]:
# get the scores and fixtures table
chelsea_data = driver.page_source
chelsea_matches = pd.read_html(chelsea_data, match='Scores & Fixtures')
chelsea_matches[0].head()

/tmp/ipython-input-562704875.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  chelsea_matches = pd.read_html(chelsea_data, match='Scores & Fixtures')


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-18,16:30 (15:30),Premier League,Matchweek 1,Sun,Home,L,0,2,Manchester City,1.0,0.8,48,39818,Enzo Fernández,4-2-3-1,3-2-4-1,Anthony Taylor,Match Report,NaN
1,2024-08-22,20:00 (19:00),Conf Lg,Play-off round,Thu,Home,W,2,0,ch Servette FC,NaN,NaN,NaN,37902,Moisés Caicedo,4-3-3,4-2-3-1,Jérémie Pignard,Match Report,Leg 1 of 2
2,2024-08-25,14:00 (13:00),Premier League,Matchweek 2,Sun,Away,W,6,2,Wolves,1.6,1.9,60,31235,Enzo Fernández,4-2-3-1,4-2-3-1,Darren England,Match Report,NaN
3,2024-08-29,20:30 (18:30),Conf Lg,Play-off round,Thu,Away,L,1,2,ch Servette FC,NaN,NaN,NaN,28000,Enzo Fernández,4-2-3-1,4-4-1-1,Marco Di Bello,Match Report,Leg 2 of 2; Chelsea won
4,2024-09-01,13:30 (12:30),Premier League,Matchweek 3,Sun,Home,D,1,1,Crystal Palace,2.4,0.5,62,39298,Enzo Fernández,4-2-3-1,3-4-3,Jarred Gillett,Match Report,NaN


In [124]:
soup = BeautifulSoup(chelsea_data)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting' in l]
links[0]

'/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/shooting/Chelsea-Match-Logs-All-Competitions'

In [125]:
driver.get(f'https://fbref.com{links[0]}')
time.sleep(5)

In [126]:
chels_shooting = driver.page_source
chels_shooting_data = pd.read_html(chels_shooting, match='Shooting')[0]
chels_shooting_data

/tmp/ipython-input-2168596512.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  chels_shooting_data = pd.read_html(chels_shooting, match='Shooting')[0]


For Chelsea                                                               \
          Date           Time            Comp           Round  Day    Venue   
0   2024-08-18  16:30 (15:30)  Premier League     Matchweek 1  Sun     Home   
1   2024-08-22  20:00 (19:00)         Conf Lg  Play-off round  Thu     Home   
2   2024-08-25  14:00 (13:00)  Premier League     Matchweek 2  Sun     Away   
3   2024-08-29  20:30 (18:30)         Conf Lg  Play-off round  Thu     Away   
4   2024-09-01  13:30 (12:30)  Premier League     Matchweek 3  Sun     Home   
..         ...            ...             ...             ...  ...      ...   
63  2025-05-11  12:00 (11:00)  Premier League    Matchweek 36  Sun     Away   
64  2025-05-16  20:15 (19:15)  Premier League    Matchweek 37  Fri     Home   
65  2025-05-25  16:00 (15:00)  Premier League    Matchweek 38  Sun     Away   
66  2025-05-28  21:00 (19:00)         Conf Lg           Final  Wed  Neutral   
67         NaN            NaN             NaN             NaN  NaN      NaN   

                                      ... Standard               Expected  \
   Result   GF   GA         Opponent  ...     Dist   FK PK PKatt       xG   
0       L    0    2  Manchester City  ...     17.1    0  0     0      1.0   
1       W    2    0   ch Servette FC  ...      NaN  NaN  1     1      NaN   
2       W    6    2           Wolves  ...     15.4    1  0     0      1.6   
3       L    1    2   ch Servette FC  ...      NaN  NaN  1     1      NaN   
4       D    1    1   Crystal Palace  ...     15.9    1  0     0      2.4   
..    ...  ...  ...              ...  ...      ...  ... ..   ...      ...   
63      L    0    2    Newcastle Utd  ...     17.3    0  0     0      0.7   
64      W    1    0   Manchester Utd  ...     19.3    1  0     0      0.8   
65      W    1    0  Nott'ham Forest  ...     14.3    0  0     0      1.1   
66      W    4    1         es Betis  ...     16.9    0  0     0      1.3   
67     --  NaN  NaN              NaN  ...     16.6   19  8    11      NaN   

                                Unnamed: 25_level_0  
   npxG npxG/Sh    G-xG np:G-xG        Match Report  
0   1.0    0.10    -1.0    -1.0        Match Report  
1   NaN     NaN     NaN     NaN        Match Report  
2   1.6    0.12    +4.4    +4.4        Match Report  
3   NaN     NaN     NaN     NaN        Match Report  
4   2.4    0.18    -1.4    -1.4        Match Report  
..  ...     ...     ...     ...                 ...  
63  0.7    0.07    -0.7    -0.7        Match Report  
64  0.8    0.07    +0.2    +0.2        Match Report  
65  1.1    0.18    -0.1    -0.1        Match Report  
66  1.3    0.12    +2.7    +2.7        Match Report  
67  NaN    0.12  +115.0  +107.0                 NaN  

[68 rows x 26 columns]

Data Cleaning

In [127]:
# we want to get rid of the multi level index
chels_shooting_data.columns = chels_shooting_data.columns.droplevel()
chels_shooting_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-18,16:30 (15:30),Premier League,Matchweek 1,Sun,Home,L,0,2,Manchester City,...,17.1,0,0,0,1.0,1.0,0.10,-1.0,-1.0,Match Report
1,2024-08-22,20:00 (19:00),Conf Lg,Play-off round,Thu,Home,W,2,0,ch Servette FC,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Match Report
2,2024-08-25,14:00 (13:00),Premier League,Matchweek 2,Sun,Away,W,6,2,Wolves,...,15.4,1,0,0,1.6,1.6,0.12,+4.4,+4.4,Match Report
3,2024-08-29,20:30 (18:30),Conf Lg,Play-off round,Thu,Away,L,1,2,ch Servette FC,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Match Report
4,2024-09-01,13:30 (12:30),Premier League,Matchweek 3,Sun,Home,D,1,1,Crystal Palace,...,15.9,1,0,0,2.4,2.4,0.18,-1.4,-1.4,Match Report


In [128]:
chels_shooting_data['Date']

,Date
0,2024-08-18
1,2024-08-22
2,2024-08-25
3,2024-08-29
4,2024-09-01
...,...
63,2025-05-11
64,2025-05-16
65,2025-05-25
66,2025-05-28


In [129]:
chelsea_matches[0].shape

(62, 20)

In [130]:
chels_shooting_data.shape

(68, 26)

In [131]:
# merge two data frames
chelsea_data = chelsea_matches[0].merge(chels_shooting_data[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
chelsea_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-18,16:30 (15:30),Premier League,Matchweek 1,Sun,Home,L,0,2,Manchester City,...,3-2-4-1,Anthony Taylor,Match Report,NaN,10,3,17.1,0,0,0
1,2024-08-22,20:00 (19:00),Conf Lg,Play-off round,Thu,Home,W,2,0,ch Servette FC,...,4-2-3-1,Jérémie Pignard,Match Report,Leg 1 of 2,NaN,NaN,NaN,NaN,1,1
2,2024-08-25,14:00 (13:00),Premier League,Matchweek 2,Sun,Away,W,6,2,Wolves,...,4-2-3-1,Darren England,Match Report,NaN,14,8,15.4,1,0,0
3,2024-08-29,20:30 (18:30),Conf Lg,Play-off round,Thu,Away,L,1,2,ch Servette FC,...,4-4-1-1,Marco Di Bello,Match Report,Leg 2 of 2; Chelsea won,NaN,NaN,NaN,NaN,1,1
4,2024-09-01,13:30 (12:30),Premier League,Matchweek 3,Sun,Home,D,1,1,Crystal Palace,...,3-4-3,Jarred Gillett,Match Report,NaN,13,7,15.9,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2025-05-08,20:00 (19:00),Conf Lg,Semi-finals,Thu,Home,W,1,0,se Djurgården,...,4-3-3,João Pinheiro,Match Report,Leg 2 of 2; Chelsea won,17,5,16.4,0,0,0
78,2025-05-11,12:00 (11:00),Premier League,Matchweek 36,Sun,Away,L,0,2,Newcastle Utd,...,3-4-3,John Brooks,Match Report,NaN,10,3,17.3,0,0,0
79,2025-05-16,20:15 (19:15),Premier League,Matchweek 37,Fri,Home,W,1,0,Manchester Utd,...,3-4-3,Chris Kavanagh,Match Report,NaN,11,3,19.3,1,0,0
80,2025-05-25,16:00 (15:00),Premier League,Matchweek 38,Sun,Away,W,1,0,Nott'ham Forest,...,4-3-1-2,Anthony Taylor,Match Report,NaN,6,2,14.3,0,0,0


In [132]:
chelsea_data.shape

(82, 26)

Scrape Data From All Teams Across Seasons

In [133]:
async def fetch_page_source(session, url):
    async with session.get(url) as response:
        return await response.text()

In [134]:
async def scrape_team_data(session, team_url, year):
    # Get the team's match data
    page_data = await fetch_page_source(session, team_url)
    matches = pd.read_html(page_data, match='Scores & Fixtures')

    # Get the link to the shooting data
    soup = BeautifulSoup(page_data, 'html.parser')
    links = [l.get("href") for l in soup.find_all('a') if l and 'all_comps/shooting' in l]

    if not links:
        print(f"No shooting data found for {team_url} in {year}")
        return None

    shooting_url = f'https://fbref.com{links[0]}'
    shooting_data = await fetch_page_source(session, shooting_url)
    shooting = pd.read_html(shooting_data, match='Shooting')[0]
    shooting.columns = shooting.columns.droplevel()

    try:
        team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError as e:
        print(f"Error merging data for {team_url} in {year}: {e}")
        return None

    team_data = team_data[team_data['Comp'] == 'Premier League']
    team_data["Season"] = year
    team_data['Team'] = team_url.split("/")[-1].replace("-Stats", "")
    return team_data

In [139]:
async def main():
    years = list(range(2025, 2020, -1))
    all_matches = []
    standings_url = "https://fbref.com/en/comps/9/2024-2025/2024-2025-Premier-League-Stats"

    # Re-initialize WebDriver before the loop
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)

    for year in years:
        print(f"Scraping data for the {year} season...")
        try:
            # Get the standings page for the current year using Selenium
            driver.get(standings_url)
            time.sleep(5) # Give the page time to load

            # Find the link for the previous season's standings for the next iteration
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            prev_standings_link_element = soup.select_one('a.prev') # Use select_one as we only need one link
            if prev_standings_link_element:
                prev_standings_link = prev_standings_link_element.get('href')
                standings_url = f'https://fbref.com{prev_standings_link}'
            else:
                print(f"Could not find link to previous season's standings for {year}. Stopping.")
                break # Stop if we can't find the link to the previous season

            # Extract team URLs from the standings page
            standings_table = soup.select('table.stats_table')
            if not standings_table:
                 print(f"No standings table found for {year}. Skipping year.")
                 continue

            a_tags = standings_table[0].find_all('a')
            links = [l.get("href") for l in a_tags]
            links = [l for l in links if '/squads' in l]
            team_urls = [f"https://fbref.com{l}" for l in links]

            for team_url in team_urls:
                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # Clean up team name

                print(f"  Scraping data for {team_name}...")
                try:
                    # Get the team's match data using Selenium
                    driver.get(team_url)
                    time.sleep(5) # Give the page time to load

                    team_page_source = driver.page_source
                    matches = pd.read_html(team_page_source, match='Scores & Fixtures')

                    soup = BeautifulSoup(team_page_source, 'html.parser')
                    links = soup.find_all('a')
                    links = [l.get("href") for l in links]
                    shooting_links = [l for l in links if l and 'all_comps/shooting' in l]

                    if not shooting_links:
                        print(f"    No shooting data link found for {team_name} in {year}. Skipping team.")
                        continue

                    # Get the team's shooting data using Selenium
                    shooting_url = f'https://fbref.com{shooting_links[0]}'
                    driver.get(shooting_url)
                    time.sleep(5) # Give the page time to load

                    shooting_page_source = driver.page_source
                    shooting = pd.read_html(shooting_page_source, match='Shooting')

                    if not shooting:
                        print(f"    No shooting table found for {team_name} in {year}. Skipping team.")
                        continue

                    shooting_df = shooting[0]
                    shooting_df.columns = shooting_df.columns.droplevel()

                    # Merge match data and shooting data
                    try:
                        team_data = matches[0].merge(shooting_df[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
                    except ValueError as e:
                        print(f"    Error merging data for {team_name} in {year}: {e}. Skipping team.")
                        continue

                    # Filter for Premier League matches and add Season and Team columns
                    team_data = team_data[team_data['Comp'] == 'Premier League']
                    team_data["Season"] = year
                    team_data['Team'] = team_name
                    all_matches.append(team_data)

                except Exception as e:
                    print(f"    An error occurred during scraping for {team_name} in {year}: {e}. Skipping team.")
                    # Continue to the next team on error

        except Exception as e:
            print(f"An error occurred during scraping for year {year}: {e}")
            print("Attempting to re-initialize WebDriver and continue to the next year...")
            try:
                driver.quit()
            except:
                pass # Ignore errors if driver is already quit
            # Re-initialize the WebDriver
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            driver = webdriver.Chrome(options=chrome_options)
            # Continue to the next year, hoping re-initialization helps
            continue

    # Concatenate all dataframes and save to CSV
    if all_matches:
        match_df = pd.concat(all_matches)
        match_df.to_csv("/content/drive/My Drive/Summer 2025/ML Soccer/matches.csv", index=False)
        print("\nScraping complete. Data saved to matches.csv")
    else:
        print("\nScraping complete. No data was collected.")

    # Close the browser after the scraping is done
    driver.quit()

In [140]:
await main()

Scraping data for the 2025 season...
  Scraping data for Liverpool...


/tmp/ipython-input-3598335593.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_page_source, match='Scores & Fixtures')
/usr/local/lib/python3.11/dist-packages/bs4/__init__.py:1032: RuntimeWarning: coroutine 'main' was never awaited
  tag = tag_class(
/tmp/ipython-input-3598335593.py:68: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(shooting_page_source, match='Shooting')


  Scraping data for Arsenal...


/tmp/ipython-input-3598335593.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_page_source, match='Scores & Fixtures')
/tmp/ipython-input-3598335593.py:68: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(shooting_page_source, match='Shooting')


  Scraping data for Manchester City...


...

  Scraping data for West Bromwich Albion...


/tmp/ipython-input-3598335593.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_page_source, match='Scores & Fixtures')
/tmp/ipython-input-3598335593.py:68: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(shooting_page_source, match='Shooting')


  Scraping data for Sheffield United...


/tmp/ipython-input-3598335593.py:51: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_page_source, match='Scores & Fixtures')
/tmp/ipython-input-3598335593.py:68: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(shooting_page_source, match='Shooting')



Scraping complete. Data saved to matches.csv


In [141]:
# close the browser
driver.quit()